# EPyR Tools - Basic Data Loading

This notebook demonstrates the basic functionality for loading EPR data from Bruker spectrometers using EPyR Tools.

## What you'll learn:
- Loading BES3T (.dsc/.dta) and ESP (.par/.spc) files
- Handling 1D and 2D EPR data formats
- Basic visualization of EPR spectra
- Parameter extraction and interpretation
- Working with both real and complex data

**Compatible with EPyR Tools v0.1.2+**

## 1. Setup and Import Libraries

In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add EPyR Tools to Python path
current_dir = Path().resolve()
epyr_root = current_dir.parent.parent  # Go up two levels from notebooks to project root

if str(epyr_root) not in sys.path:
    sys.path.insert(0, str(epyr_root))
    
print(f"Looking for EPyR Tools in: {epyr_root}")
print(f"EPyR package should be at: {epyr_root / 'epyr'}")

# Check if epyr directory exists
epyr_package_dir = epyr_root / 'epyr'
if epyr_package_dir.exists():
    print(f"✅ EPyR package directory found")
else:
    print(f"❌ EPyR package directory not found")
    print(f"Please make sure you're running this notebook from examples/notebooks/")

# Import EPyR Tools
try:
    import epyr
    print(f"\n✅ EPyR Tools loaded successfully!")
    print(f"Version: {epyr.__version__}")
except ImportError as e:
    print(f"\n❌ Failed to import EPyR Tools: {e}")
    print("\nTroubleshooting:")
    print("1. Make sure you're in the examples/notebooks/ directory")
    print("2. Try: pip install -e . (from project root)")
    print("3. Check that epyr/ directory exists in project root")

## 2. Find Available Data Files

Let's look for EPR data files in our consolidated data directory.

In [ ]:
# Look for sample data files in the consolidated data directory
data_dir = Path("../data")

print(f"Looking for data files in: {data_dir.resolve()}")

# Find all EPR files in the data directory
bes3t_files = list(data_dir.glob("*.DSC")) + list(data_dir.glob("*.dsc"))
esp_files = list(data_dir.glob("*.par"))

print(f"\nFound {len(bes3t_files)} BES3T files and {len(esp_files)} ESP files")

print("\nAvailable files:")
all_files = [(f, "BES3T") for f in bes3t_files] + [(f, "ESP") for f in esp_files]

for i, (file_path, file_type) in enumerate(all_files):
    file_size = file_path.stat().st_size / 1024  # KB
    print(f"{i+1}. [{file_type}] {file_path.name} ({file_size:.1f} KB)")
    
if not all_files:
    print("❌ No EPR files found!")
    print(f"Please add your .DSC/.DTA or .PAR/.SPC files to: {data_dir.resolve()}")

## 3. Load and Analyze Each File

Now let's load each file and see what type of data we have.

In [ ]:
# Store loaded data for later use
loaded_data = []

print("EPyR Tools - Basic Data Loading Example")
print("=" * 40)

for file_format, file_path in all_files:
    print(f"\n🔄 Loading {file_format} file: {file_path.name}")
    
    try:
        # Load EPR data
        x, y, params, filepath = epyr.eprload(str(file_path), plot_if_possible=False)
        
        if x is None or y is None:
            print(f"  ❌ Failed to load data from {file_path.name}")
            continue
        
        # Handle both 1D and 2D data
        if isinstance(x, list) and len(x) > 1:
            # 2D data: x is a list of axes
            print(f"  📊 Data type: 2D")
            print(f"  📐 Data shape: {y.shape}")
            print(f"  📏 Dimensions: {len(x)} axes")
            if hasattr(x[0], '__len__'):
                print(f"  🧲 Field range: {x[0].min():.1f} to {x[0].max():.1f} G")
            # Use magnitude for complex data
            y_display = np.abs(y) if np.iscomplexobj(y) else y
            print(f"  📈 Signal range: {y_display.min():.2e} to {y_display.max():.2e}")
            print(f"  🔢 Complex data: {'Yes' if np.iscomplexobj(y) else 'No'}")
        else:
            # 1D data: x is a single array
            x_array = x[0] if isinstance(x, list) else x
            print(f"  📊 Data type: 1D")
            print(f"  📏 Data points: {len(x_array)}")
            print(f"  🧲 Field range: {x_array.min():.1f} to {x_array.max():.1f} G")
            print(f"  📈 Signal range: {y.min():.2e} to {y.max():.2e}")
            print(f"  🔢 Complex data: {'Yes' if np.iscomplexobj(y) else 'No'}")
        
        # Display key parameters
        key_params = {
            "MWFQ": "Microwave Frequency (Hz)",
            "MWPW": "Microwave Power (dB)",
            "AVGS": "Number of Averages",
            "HCF": "Center Field (G)",
            "HSW": "Sweep Width (G)",
            "MF": "Frequency (GHz)",
            "MP": "Power",
            "MA": "Modulation Amplitude (G)",
            "RCT": "Time Constant (s)"
        }
        
        print("  📋 Key Parameters:")
        found_params = 0
        for param, description in key_params.items():
            if param in params:
                value = params[param]
                print(f"    • {description}: {value}")
                found_params += 1
        
        if found_params == 0:
            print("    • No standard parameters found")
        
        # Store for plotting
        loaded_data.append({
            'file_path': file_path,
            'file_format': file_format,
            'x': x,
            'y': y,
            'params': params,
            'is_2d': isinstance(x, list) and len(x) > 1,
            'is_complex': np.iscomplexobj(y)
        })
        
        print(f"  ✅ Successfully loaded!")
        
    except Exception as e:
        print(f"  ❌ Error loading {file_path.name}: {e}")

print(f"\n📊 Summary: Successfully loaded {len(loaded_data)} out of {len(all_files)} files")

## 4. Visualize the Data

Now let's create plots for each loaded dataset.

In [ ]:
# Set up matplotlib for better plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("📈 Creating visualizations for loaded data...\n")

for i, data in enumerate(loaded_data):
    print(f"Plotting {i+1}/{len(loaded_data)}: {data['file_path'].name}")
    
    x = data['x']
    y = data['y']
    params = data['params']
    file_path = data['file_path']
    file_format = data['file_format']
    is_2d = data['is_2d']
    is_complex = data['is_complex']
    
    # Create plot based on data type
    if is_2d:
        # 2D data plotting
        y_plot = np.abs(y) if is_complex else y
        
        if len(y_plot.shape) == 2:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # 2D color map
            im = ax1.imshow(y_plot, aspect='auto', origin='lower', cmap='viridis')
            ax1.set_xlabel('Field Points')
            ax1.set_ylabel('Parameter Points')
            ax1.set_title(f'2D EPR Data: {file_path.stem}')
            plt.colorbar(im, ax=ax1, label='Signal (a.u.)')
            
            # Representative slices
            n_slices = min(5, y_plot.shape[0])
            colors = plt.cm.tab10(np.linspace(0, 1, n_slices))
            
            for j in range(n_slices):
                idx = j * (y_plot.shape[0] // n_slices) if n_slices > 1 else 0
                if hasattr(x[0], '__len__'):
                    ax2.plot(x[0], y_plot[idx, :], color=colors[j], 
                            alpha=0.8, linewidth=1.5, label=f'Slice {idx+1}')
                else:
                    ax2.plot(y_plot[idx, :], color=colors[j], 
                            alpha=0.8, linewidth=1.5, label=f'Slice {idx+1}')
            
            ax2.set_xlabel('Field (G)' if hasattr(x[0], '__len__') else 'Points')
            ax2.set_ylabel('EPR Signal (a.u.)')
            ax2.set_title('Representative Spectra')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            
            # Add data info
            info_text = f"Format: {file_format}\nShape: {y.shape}\nComplex: {is_complex}"
            ax1.text(0.02, 0.98, info_text, transform=ax1.transAxes,
                    verticalalignment='top', bbox=dict(boxstyle='round', 
                    facecolor='white', alpha=0.8))
        else:
            # Fallback for unexpected format
            plt.figure(figsize=(12, 6))
            plt.plot(y_plot.flatten(), "b-", linewidth=1.5)
            plt.xlabel("Data Points")
            plt.ylabel("EPR Signal (a.u.)")
            plt.title(f"EPR Data: {file_path.stem} ({file_format} format)")
            plt.grid(True, alpha=0.3)
    else:
        # 1D data plotting
        plt.figure(figsize=(12, 6))
        x_array = x[0] if isinstance(x, list) else x
        plt.plot(x_array, y, "b-", linewidth=1.5)
        plt.xlabel("Magnetic Field (G)")
        plt.ylabel("EPR Signal (a.u.)")
        plt.title(f"EPR Spectrum: {file_path.stem} ({file_format} format)")
        plt.grid(True, alpha=0.3)
        
        # Add info box
        info_text = f"Points: {len(x_array)}\nRange: {x_array.max()-x_array.min():.0f} G\nFormat: {file_format}"
        plt.text(0.02, 0.98, info_text, transform=plt.gca().transAxes,
                 verticalalignment='top', bbox=dict(boxstyle='round', 
                 facecolor='white', alpha=0.8))
    
    # Add frequency info if available
    freq = params.get("MWFQ", params.get("MF", None))
    if freq:
        if isinstance(freq, str):
            try:
                freq_ghz = float(freq)
            except:
                freq_ghz = None
        else:
            freq_ghz = freq / 1e9 if freq > 1e6 else freq
        
        if freq_ghz is not None:
            # Add to appropriate axis
            current_ax = plt.gca()
            if is_2d and len(y.shape) == 2:
                # For 2D plots, add to the second subplot
                current_ax = plt.subplot(1, 2, 2)
            
            current_ax.text(
                0.98, 0.02, f"Frequency: {freq_ghz:.3f} GHz",
                transform=current_ax.transAxes,
                verticalalignment='bottom', horizontalalignment='right',
                bbox=dict(boxstyle="round", facecolor="yellow", alpha=0.8),
            )
    
    plt.tight_layout()
    plt.show()

print(f"\n✅ All plots created successfully!")

## 5. Data Analysis Examples

Let's perform some basic analysis on the loaded data.

In [ ]:
print("📊 Basic EPR Data Analysis")
print("=" * 30)

for i, data in enumerate(loaded_data):
    print(f"\n🔍 Analysis of: {data['file_path'].name}")
    print("-" * 50)
    
    x = data['x']
    y = data['y']
    is_2d = data['is_2d']
    is_complex = data['is_complex']
    
    if is_2d:
        print(f"  📐 Data type: 2D ({y.shape})")
        print(f"  🔢 Complex: {'Yes' if is_complex else 'No'}")
        
        # Use magnitude for analysis if complex
        y_analysis = np.abs(y) if is_complex else y
        
        print(f"  📈 Signal statistics:")
        print(f"    • Mean: {np.mean(y_analysis):.2e}")
        print(f"    • Std: {np.std(y_analysis):.2e}")
        print(f"    • Min: {np.min(y_analysis):.2e}")
        print(f"    • Max: {np.max(y_analysis):.2e}")
        print(f"    • Peak-to-peak: {np.ptp(y_analysis):.2e}")
        
        # Find strongest signal position
        max_pos = np.unravel_index(np.argmax(np.abs(y_analysis)), y_analysis.shape)
        print(f"  🎯 Strongest signal at position: {max_pos}")
        
        # Field information if available
        if hasattr(x[0], '__len__'):
            field_at_max = x[0][max_pos[1]] if max_pos[1] < len(x[0]) else "N/A"
            print(f"  🧲 Field at strongest signal: {field_at_max} G")
            print(f"  📏 Field range: {x[0].min():.1f} - {x[0].max():.1f} G")
        
    else:
        print(f"  📐 Data type: 1D ({len(y)} points)")
        print(f"  🔢 Complex: {'Yes' if is_complex else 'No'}")
        
        x_array = x[0] if isinstance(x, list) else x
        
        print(f"  📈 Signal statistics:")
        print(f"    • Mean: {np.mean(y):.2e}")
        print(f"    • Std: {np.std(y):.2e}")
        print(f"    • Min: {np.min(y):.2e}")
        print(f"    • Max: {np.max(y):.2e}")
        print(f"    • Peak-to-peak: {np.ptp(y):.2e}")
        print(f"    • SNR estimate: {np.ptp(y)/np.std(y):.1f}")
        
        # Find peaks and troughs
        max_idx = np.argmax(np.abs(y))
        max_field = x_array[max_idx]
        print(f"  🎯 Strongest signal at: {max_field:.1f} G")
        print(f"  🧲 Field range: {x_array.min():.1f} - {x_array.max():.1f} G")
        print(f"  📏 Field resolution: {(x_array.max()-x_array.min())/(len(x_array)-1):.2f} G/point")
    
    # Parameter summary
    params = data['params']
    print(f"  📋 Available parameters: {len(params)} total")
    
    # Show microwave frequency in GHz if available
    freq = params.get("MWFQ", params.get("MF", None))
    if freq:
        if isinstance(freq, str):
            try:
                freq_val = float(freq)
            except:
                freq_val = None
        else:
            freq_val = freq
        
        if freq_val:
            freq_ghz = freq_val / 1e9 if freq_val > 1e6 else freq_val
            print(f"  📡 Microwave frequency: {freq_ghz:.4f} GHz")
            
            # Calculate approximate g-factor for strongest signal
            if not is_2d and freq_ghz > 1:
                # g = hν / (μB * B), where h = 6.626e-34, μB = 9.274e-24, ν in Hz, B in Tesla
                h = 6.626e-34  # Planck constant
                mu_b = 9.274e-24  # Bohr magneton
                b_tesla = max_field * 1e-4  # Convert Gauss to Tesla
                if b_tesla > 0:
                    g_factor = (h * freq_ghz * 1e9) / (mu_b * b_tesla)
                    print(f"  🔬 Approximate g-factor at peak: {g_factor:.3f}")

print(f"\n✅ Analysis complete for all {len(loaded_data)} datasets!")

## 6. Export Data for Further Analysis

Let's show how to export the data for use in other tools.

In [ ]:
import pandas as pd

print("💾 Exporting Data for External Analysis")
print("=" * 40)

# Create output directory
output_dir = Path("../data/exported")
output_dir.mkdir(exist_ok=True)

exported_files = []

for i, data in enumerate(loaded_data):
    file_path = data['file_path']
    x = data['x']
    y = data['y']
    params = data['params']
    is_2d = data['is_2d']
    is_complex = data['is_complex']
    
    base_name = file_path.stem
    
    print(f"\n📤 Exporting: {file_path.name}")
    
    if is_2d:
        # For 2D data, export a few representative slices
        y_export = np.abs(y) if is_complex else y
        
        # Export first 3 spectra as separate CSV files
        n_export = min(3, y_export.shape[0])
        for j in range(n_export):
            csv_file = output_dir / f"{base_name}_slice_{j+1}.csv"
            
            if hasattr(x[0], '__len__'):
                df = pd.DataFrame({
                    'Field_G': x[0],
                    'Intensity': y_export[j, :]
                })
            else:
                df = pd.DataFrame({
                    'Index': range(len(y_export[j, :])),
                    'Intensity': y_export[j, :]
                })
            
            # Add metadata as comments
            with open(csv_file, 'w') as f:
                f.write(f"# EPyR Tools Export - Slice {j+1} of {base_name}\n")
                f.write(f"# Original file: {file_path.name}\n")
                f.write(f"# Data type: 2D {'Complex' if is_complex else 'Real'}\n")
                f.write(f"# Full shape: {y.shape}\n")
                f.write(f"# Slice index: {j}\n")
                
                # Add key parameters
                freq = params.get("MWFQ", params.get("MF", None))
                if freq:
                    f.write(f"# Frequency: {freq}\n")
                
                f.write(f"#\n")
                df.to_csv(f, index=False)
            
            print(f"  ✅ Slice {j+1} saved: {csv_file.name}")
            exported_files.append(csv_file)
        
        # Also save full 2D data as numpy
        npz_file = output_dir / f"{base_name}_full_2D.npz"
        np.savez(npz_file, 
                 x_axis=x[0] if hasattr(x[0], '__len__') else np.arange(y.shape[1]),
                 y_axis=x[1] if len(x) > 1 and hasattr(x[1], '__len__') else np.arange(y.shape[0]),
                 intensity=y,
                 is_complex=is_complex,
                 original_file=str(file_path))
        print(f"  ✅ Full 2D data saved: {npz_file.name}")
        exported_files.append(npz_file)
        
    else:
        # For 1D data, export as simple CSV
        x_array = x[0] if isinstance(x, list) else x
        
        csv_file = output_dir / f"{base_name}_1D.csv"
        
        df = pd.DataFrame({
            'Field_G': x_array,
            'Intensity': y
        })
        
        # Add metadata as comments
        with open(csv_file, 'w') as f:
            f.write(f"# EPyR Tools Export - 1D EPR Spectrum\n")
            f.write(f"# Original file: {file_path.name}\n")
            f.write(f"# Data points: {len(x_array)}\n")
            f.write(f"# Field range: {x_array.min():.1f} - {x_array.max():.1f} G\n")
            
            # Add key parameters
            for param, value in params.items():
                if param in ['MWFQ', 'MWPW', 'HCF', 'HSW', 'MF', 'MP']:
                    f.write(f"# {param}: {value}\n")
            
            f.write(f"#\n")
            df.to_csv(f, index=False)
        
        print(f"  ✅ 1D data saved: {csv_file.name}")
        exported_files.append(csv_file)

print(f"\n📁 All exports saved to: {output_dir.resolve()}")
print(f"📋 Total files exported: {len(exported_files)}")

print(f"\n💡 Usage examples:")
print(f"  Python: df = pd.read_csv('filename.csv', comment='#')")
print(f"  R: data <- read.csv('filename.csv')")
print(f"  Excel: File → Open → filename.csv")
print(f"  Origin/Igor: Import CSV with field separation")

## Summary

🎉 **Congratulations!** You've successfully completed the basic EPR data loading tutorial!

### What we accomplished:

✅ **Loaded EPR data** from both BES3T (.dsc/.dta) and ESP (.par/.spc) formats  
✅ **Handled different data types** including 1D spectra and 2D datasets  
✅ **Managed complex data** by taking magnitude for visualization  
✅ **Extracted key parameters** like frequency, power, and field settings  
✅ **Created professional visualizations** with proper axis labels and info boxes  
✅ **Performed basic analysis** including SNR estimation and g-factor calculation  
✅ **Exported data** in formats compatible with other analysis tools  

### Next Steps:

- **Baseline Correction**: Remove drift and artifacts from your spectra
- **Advanced Analysis**: Peak fitting, integration, and quantitative analysis
- **FAIR Data Conversion**: Convert to open formats for better accessibility
- **Batch Processing**: Automate analysis of multiple files

### Key Functions Learned:

- `epyr.eprload()` - Main function for loading EPR data
- Handling both 1D arrays and 2D lists of axes
- Complex data visualization using `np.abs()`
- Parameter extraction and interpretation

### Need Help?

- Check the other example notebooks for more advanced features
- Look at the example scripts in `examples/scripts/`
- Visit the project documentation for API reference